# SET: Kaggle Voice Recognition
1. 캐글 '마크업' 이 구려서, 이미지 리사이징 옵션이 안먹네요.. 어쩔수 없이 하단에 **HTML** 사용

<img src=https://www.skylinelabs.in/blog/images/tensorflow.jpg align=left width=450>

# 관련문서 참조
> 1. [Kaggle.com 원본: BOSTON 의 집값 예측 :](https://www.kaggle.com/sunmiyoon/one-hour-analysis-written-in-both-eng-and-kor/comments#290148)
* [Kaggle.com 원본: 남/녀 목소리 판별 :](https://www.kaggle.com/primaryobjects/voicegender)
* [Pandas Documentation (**10 minutes to Pandas** ) :](https://pandas.pydata.org/pandas-docs/stable/10min.html)
* [ **판다스** , 10분 만에 둘러보기 (한글번역) :](https://dandyrilla.github.io/2017-08-12/pandas-10min/)
***

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

In [2]:
# 데이터 프레임의 형태부터 파악 합니다.
df = pd.read_csv('../input/voice.csv')
print(df.columns, len(df.columns) )

In [3]:
print("총 {:,} 개의 데이터".format(len(df)))       # 3,168개 데이터
df.head(2)
df.tail(2)
# 데이터 20갸, 레이블 1개 (21)번째 칼럼 = male ... female
# 데이터 값의 분포를 봅니다.
# 마지막 '라벨'값이 숫자가 아니라 '스트링' (Male/Female) 입니다. 숫자 1,0 으로 바꿔줍니다.

In [4]:
# '라벨'값을 숫자로 바꾼다, ---> male = 1.0, female = 0.0
if 'female' in set(df['label']) or 'male' in set(df['label']):
    print('[변경전] %s' % set(df['label']), end='')
    df['label'].replace(['male', 'female'], [1., 0.], inplace=True)
    print('  .... [변경후] %s' % set(df['label']))
else: 
    print('이미, 값을 변경 하였습니다. 라벨은 ... %s' % set(df['label']))

df.tail(2)

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

print(list(df['label']).count(1.0) == list(df['label']).count(0.0))
print("(남성):{:,} 개 + (여성):{:,}게 = 총 {:,} 개의 데이터".format(
    list(df['label']).count(1.0), 
    list(df['label']).count(0.0), 
    len(df)))
f, ax = plt.subplots(figsize=(20,5))
sns.distplot(df['label'])

In [6]:
# shuffling -- 데이터를 무작위로 섞는다.
df = df.reindex(np.random.permutation(df.index))

df_train = df[:2168]      #  0 ~ 2167 개 = 학습 세트
df_test = df[2168:]       #  1000 개 = 테스트세트

print()
print("학습데이터 : {:,} 개".format(len(df_train)))
print("검증데이터 : {:,} 개".format(len(df_test)))

df_train.describe()

In [7]:
df_test.describe()

## Data scanning
데이터 스캔

---

* 분석 전에 날것의 데이터를 그대로 스캔하는 것이 중요합니다. 어떤 변수들이 들어있고, 각자의 데이터 타입은 무엇인지, 어떤 변수는 dummy로 사용하고 어떤 변수는 numerical 로 사용 할 것인지 염두해 두면서 데이터를 스캔합니다.

In [8]:
df_train.tail(2)
# print(test.head(2))
# print(v_data.tail(2))
# print(test.tail(2))
# v_data.size
# v_data.columns

## Summary Statistics
요약 통계

---

In [10]:
print(df_train.describe())

### Correlation matrix (상관관계 매트릭스)
* 첫번째로, 상관관계 테이블을 이용하여 변수들간의 상관관계를 살펴봅니다. 만약 독립변수(Xs)들 간에 상관관계가 높다면 회귀분석의 결과(coefficient)를 신뢰하기가 힘들어 집니다. 예를 들면, A라는 변수와 B라는 두 독립변수의 상관관계가 매우 높다면 A의 계수를 온전히 A가 Y에 미치는 영향이라고 해석하기 어렵습니다. 

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

f, ax = plt.subplots(figsize=(6,5))
sns.heatmap(df_train.corr(), vmax=.8, square=True);

* 두 변수의 상관관계가 높은 것이 히트맵에서는 하얀 칸으로 표시됩니다. 상관관계가 매우 높은 페어는 다음과 같습니다.
 * meanfreq - median
 * meanfreq - Q25
 * meanfreq - Q75
 * meanfreq - mode
 * meanfreq - centroid
 * sd - IQR
 * sd - sfm
 * median - Q25
 * median - Q75
 * median - mode
 * median - centroid
 * centroid - median
 * Q25 - centroid
 * Q75 - centroid
 * skew - kurt
 * sp.ent - sfm
 * mode - centroid
 * meandom - maxdom
 * meandom - dfrange
 * maxdom - dfrange
 
 각 페어 당 label과 상관관계가 더 높은 변수 하나만 포함시킵니다.
* 특정 변수가 Y에 미치는 영향을 정확하게 설명하고 싶다면 이 과정이 필수적입니다. 하지만 단지 Y를 잘 예측하는 것이 모델의 목적이라면 굳이 이 단계에서 변수를 제거 할 필요는 없습니다.

In [15]:
def show_bigger(f1, f2):
    print('{} > {}'.format(stronger_relation(f1, f2)[0], stronger_relation(f1, f2)[1]))

def stronger_relation(f1, f2):       # 헬퍼()
    f1_corr = df_train.corr().loc[f1,'label']
    f2_corr = df_train.corr().loc[f2,'label']
    # print(f1_corr, f2_corr)
    return (f1, f2) if (f1_corr >= f2_corr) else (f2, f1)

In [17]:
show_bigger('meanfreq', 'median')    # median > meanfreq
show_bigger('meanfreq', 'Q25')       # meanfreq > Q25
show_bigger('meanfreq', 'Q75')       # Q75 > meanfreq
show_bigger('meanfreq', 'mode')      # mode > meanfreq
show_bigger('meanfreq', 'centroid')  # meanfreq > centroid

# Q75 > mode > median > meanfreq > centroid > Q25
#   ... 이 계열에선, Q75만 포함시키고 나머지는 드롭(Drop:제거) 한다
#   ... 유사한 영향력을 발휘하는 인자 들, 중에, Q75의 영향력이 가장 크기 때문이다

show_bigger('sd', 'IQR')
show_bigger('sd', 'sfm')

show_bigger('median', 'Q25')
show_bigger('median', 'Q75')
show_bigger('median', 'mode')
show_bigger('median', 'centroid')

show_bigger('Q25', 'centroid')
show_bigger('Q75', 'centroid')
show_bigger('mode', 'centroid')

show_bigger('skew', 'kurt')
show_bigger('sp.ent', 'sfm')

show_bigger('meandom', 'maxdom')
show_bigger('meandom', 'dfrange')

show_bigger('maxdom', 'dfrange')

show_bigger('mode', 'Q75')
show_bigger('IQR', 'sp.ent')


# **영향력**이 큰 **지표**만 선택
영향력 **상위 지표 5개** (Q75, IQR, kurt, sp.ent, dfrange)만 남기고
나머지 비슷한 영향력을 발휘하는 인자들은 **대부분 'Drop'(제거)**한다.
***
> * **Q75** > mode > median > meanfreq > centroid > Q25    
> * **IQR** > sd > sfm    
> * **kurt** > skew     
> * **sp.ent** > sfm     
> * **dfrange** > maxdom > meandom     

## **제거 리스트 (10개)** :
> [mode, median, meanfreq, centroid, Q25, sd, skew, sfm, maxdom, meandom]


In [ ]:
df_train = df_train.drop(['sp.ent', 'mode', 'meanfreq', 'centroid', 'median', 'Q25', 'sd', 'sfm', 'skew', 'sfm', 'dfrange', 'maxdom'], axis=1)
df_test = df_test.drop(['sp.ent', 'mode', 'meanfreq', 'centroid', 'median', 'Q25', 'sd', 'sfm', 'skew', 'sfm', 'dfrange', 'maxdom'], axis=1)

In [18]:
print(len(df_train.columns), df_train.columns)
# 11개를 제거하고 10개 '컬럼'만 남았음..

In [20]:
f, ax = plt.subplots(figsize=(5,4))
sns.heatmap(df_train.corr(), vmax=.8, square=True)

## Histogram and Scatter chart
히스토그램과 스캐터 차트

---

* 먼저, 가장 중요한 gender 데이터를 확인 해 봐야 합니다. gender에 음수는 없는지, 말도 안되게 큰 값은 없는지 체크하면서 데이터가 과연 활용할만한 데이터인지 살펴봐야 합니다. 물론 캐글은 깨끗한 데이터를 제공하니 이런 문제가 많지 않지만, 웹 크롤링을 얻은 데이터나 실생활 또는 업무에서 만날 수 있는 데이터들은 이처럼 깨끗하지 않습니다. 아웃라이어를 처리하는 방법에는 winsorization, truncation 등이 있습니다. 

In [23]:
%matplotlib inline
import matplotlib.pyplot as plt

# I think this graph is more elegant than pandas.hist()
# train['SalePrice'].hist(bins=100)
sns.distplot(df_train['kurt'])

### Scatter chart
산점도, 스캐터 차트
* Y축은 모두 gender이고, 독립변수를 X축에 맞춰 모든 변수에 대해 산점도를 그렸습니다. 데이터 시각화는 보기에 멋있어 보이기 위해 하는 것만은 아닙니다. 이렇게 모든 독립변수들에 대해 산점도를 뿌리면 어떤 변수가 특히 종속변수과 연관이 있는지, 다시 말하면 얼마나 종속변수를 설명해 주는지 한 눈에 볼 수 있습니다. 예를 들어, OverallQual과 SalePrice가 양의 관계를 보이며 산점도가 우상향하는 형태를 보이는 것을 확인 할 수 있습니다. 데이터가 수평에 가깝게 뿌려져 있다면 그 변수는 gender 와 낮은 관계를 가진다고 해석할 수도 있습니다. 하지만 이 데이터셋의 경우에는 변수의 갯수도 많고, 변수들간 관계가 복잡하기 때문에 회귀분석을 하기 이전에 성급하게 결론을 내려서는 안 됩니다.

In [26]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 6, figsize=(15, 7), sharey=True)
for col, a in zip(df_train.columns, axes.flatten()):
    if col == 'label':   
        a.set_title(col)
        a.scatter(df['label'], df['label'])
    else:
        df = df_train[['label', col]].dropna()
        a.set_title(col)
        a.scatter(df[col], df['label'])

* 이 data를 lab5-ex.ipynb에 적용해보자.

In [29]:
# Lab 5 Logistic Regression Classifier
import tensorflow as tf
import numpy as np
tf.set_random_seed(743)  # for reproducibility

# collect data
x_data = df_train.loc[:,['Q75','IQR','kurt','sp.ent']].values
y_data = df_train.loc[:,['label']].values

In [30]:
print(x_data[0],y_data[0])
#len(x_data)
type(x_data)
#y_data

* build a model

In [31]:
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([4, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

* train a model

In [33]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2000):
    cost_val, _ = sess.run([cost, optimizer], feed_dict={X: x_data, Y: y_data})
    if step % 10 == 0:
        print(step, cost_val)

x_test = df_test.loc[:,['Q75','IQR','kurt','sp.ent']].values
y_test = df_test.loc[:,['label']].values
h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_test, Y: y_test})
print("Accuracy: ", a)
print(c[0:8], y_test[0:8])